# Udacity Project P3: Wrangle OpenStreetMap Data

## Introduction

I used an OpenStreetMap export of the area of Paderborn, the place where I live.

## Data audit and inconsistencies

Before converting the OpenStreetMap XML file to a JSON, I audited the data for inconsistencies.

#### Street adresses
The street adresses looked fine to me. German street have a large variety of endings. The most common is "Straße" and "Weg". However, there are countless more and sometimes they are written with a "-", sometimes with a space and sometimes as a single word, so it's nearly impossible to check whether this is a valid name or not. I checked if there were any abbreviations but couldn't find any in the dataset.

#### Postocdes
Then I checked the postcodes: all postcodes should have 5 digits and start with 3 or 5, which is correct for Paderborn and the surrounding areas.

In [7]:
import os
import xml.etree.cElementTree as ET
from collections import defaultdict

osmfile = "paderborn.osm"
osmjsonfile = "paderborn.osm.json"

for _, element in ET.iterparse(osmfile):
    if element.tag == "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "addr:postcode":
                if len(tag.attrib['v']) != 5:
                    print(tag.attrib['v'])
                if tag.attrib['v'][0] != "3" and tag.attrib['v'][0] != "5":
                    print(tag.attrib['v'])


85053


So there is obviously one wrong postcode in the data. However, when I checked the entry I found that this is the address of a chainstore where someone entered the address of the central office.

#### Phone numbers
The phone numbers were really inconsistent. Some had the country code for Germany, so they were starting with "+49" or "0049". The continuation after this code was also inconsistent: some included the following "0", some did not.
There were also some mobile phone numbers (beginning with 017..., for example) and some service numbers (starting with 0800...).

I also noticed that there were some "emergency_access_point" in the dataset which had set the phone number "112", so in order to leave these numbers as they are, I also added that.

In [ ]:
def update_phone(phone_number):
    # http://stackoverflow.com/questions/6116978/python-replace-multiple-strings

    rep = {"-": "", "(": "", ")": "", "/": "", " ": "","-": "", "+": "", ".": ""}
    rep = dict((re.escape(k), v) for k, v in rep.iteritems())
    pattern = re.compile("|".join(rep.keys()))
    phone_number = pattern.sub(lambda m: rep[re.escape(m.group(0))], phone_number)

    if phone_number[:2] == "49":
         phone_number = "0" + phone_number[2:]

    if phone_number[:4] == "0049":
         phone_number = phone_number[4:]

    if phone_number[:2] == "00":
         phone_number = phone_number[1:]

    if phone_number[0] != "0" and phone_number != "112":
        phone_number = "0" + phone_number

    return phone_number

## Explore the data
### Overview

In [8]:
from pprint import pprint
from pymongo import MongoClient

client = MongoClient('localhost:27017')
db = client["udacity"]
paderborn = db.paderborn_osm

In [9]:
print("Filesizes:")
print('The original OSM file is {} MB'.format(os.path.getsize(osmfile)/1.0e6))
print('The original JSON file is {} MB'.format(os.path.getsize(osmjsonfile)/1.0e6))


Filesizes:
The original OSM file is 310.451692 MB
The original JSON file is 334.587118 MB


In [10]:
print("Users: {}".format(len(paderborn.distinct("created.user"))))

Users: 906


In [11]:
print("Number of documents: {}".format(paderborn.find().count()))

Number of documents: 1479054


In [12]:
print("Number of nodes:",paderborn.find({'type':'node'}).count())
print("Number of ways:",paderborn.find({'type':'way'}).count())

Number of nodes: 1266754
Number of ways: 212194


## Further exploration
### Top 10 leisure

In [24]:
leisure = paderborn.aggregate([{'$match': {'leisure': {'$exists': 1}}},
                                {'$group': {'_id': '$leisure',
                                            'count': {'$sum': 1}}}, 
                                {'$sort': {'count': -1}},
                                {'$limit': 10}])
print(list(leisure))

[{'count': 557, '_id': 'pitch'}, {'count': 486, '_id': 'playground'}, {'count': 168, '_id': 'park'}, {'count': 138, '_id': 'sports_centre'}, {'count': 122, '_id': 'nature_reserve'}, {'count': 117, '_id': 'garden'}, {'count': 95, '_id': 'common'}, {'count': 44, '_id': 'picnic_table'}, {'count': 36, '_id': 'swimming_pool'}, {'count': 27, '_id': 'horse_riding'}]


### Top 10 amenities

In [13]:
amenity = paderborn.aggregate([{'$match': {'amenity': {'$exists': 1}}},
                                {'$group': {'_id': '$amenity',
                                            'count': {'$sum': 1}}}, 
                                {'$sort': {'count': -1}},
                                {'$limit': 10}])
print(list(amenity))

[{'count': 4031, '_id': 'bench'}, {'count': 1676, '_id': 'parking'}, {'count': 822, '_id': 'waste_basket'}, {'count': 675, '_id': 'vending_machine'}, {'count': 455, '_id': 'bicycle_parking'}, {'count': 374, '_id': 'recycling'}, {'count': 287, '_id': 'restaurant'}, {'count': 281, '_id': 'post_box'}, {'count': 276, '_id': 'place_of_worship'}, {'count': 250, '_id': 'shelter'}, {'count': 234, '_id': 'hunting_stand'}, {'count': 175, '_id': 'kindergarten'}, {'count': 172, '_id': 'school'}, {'count': 153, '_id': 'fast_food'}, {'count': 134, '_id': 'bank'}, {'count': 131, '_id': 'pub'}, {'count': 126, '_id': 'telephone'}, {'count': 121, '_id': 'cafe'}, {'count': 107, '_id': 'toilets'}, {'count': 92, '_id': 'water'}]


### Benches

Funny enough, the most found amenity are benches. There are quite a lot of lakes here, all surrounded by waling and bicycle paths, so this is somehow reasonable. A quick look into the data revealed that there is even more information about these benches:

#### Material

In [68]:
bench_material = paderborn.aggregate([{'$match':{'amenity': 'bench'}},
        {'$group':{'_id':'$material',
                   'count':{'$sum':1}}},
        {'$sort':{'count':-1}},
        {'$limit':3}])

print(list(bench_material))

[{'_id': 'wood', 'count': 2203}, {'_id': None, 'count': 845}, {'_id': 'metal', 'count': 753}]


#### Colour

In [48]:
bench_colour = paderborn.aggregate([{'$match':{'amenity': 'bench'}},
        {'$group':{'_id':'$colour',
                   'count':{'$sum':1}}},
        {'$sort':{'count':-1}},
        {'$limit':3}])

print(list(bench_colour))

[{'_id': 'brown', 'count': 2119}, {'_id': None, 'count': 957}, {'_id': 'green', 'count': 345}]


#### Seats

In [70]:
bench_seats = paderborn.aggregate([{'$match':{'amenity': 'bench'}},
        {'$group':{'_id':'$seats',
                   'count':{'$sum':1}}},
        {'$sort':{'count':-1}},
        {'$limit':3}])

print(list(bench_seats))

[{'_id': '3', 'count': 2642}, {'_id': None, 'count': 968}, {'_id': '4', 'count': 148}]


### Restaurant Opening hours

After information about benches was so detailed, I also checked the information about opening hours of resturants in order to find out if there were opening hours available and if yes, if they were just some kind of placeholder or detailed.

In [22]:
restaurant_opening = paderborn.aggregate([{'$match':{'amenity': 'restaurant'}},
        {'$group':{'_id':'$opening_hours',
                   'count':{'$sum':1}}},
        {'$sort':{'count':-1}},
        {'$limit':10}])

print(list(restaurant_opening))

[{'count': 158, '_id': None}, {'count': 2, '_id': 'Mo-Su 11:00-23:00'}, {'count': 2, '_id': 'Mo-Su 11:30-23:00'}, {'count': 2, '_id': 'off'}, {'count': 2, '_id': 'Mo-Su 09:00+'}, {'count': 1, '_id': 'Mo-Sa 11:30-15:00, 18:00-22:30; Su off'}, {'count': 1, '_id': 'Mo-Fr 17:00-01:00, Sa,Su,PH 09:00-01:00'}, {'count': 1, '_id': 'Tu-Fr 10:45-14:30, 17:00-23:00; Sa-Mo 16:00-23:00'}, {'count': 1, '_id': 'Tu-Su 12:00-15:00,18:00-24:00'}, {'count': 1, '_id': 'We-Su 11:30-22:30; Mo-Tu off'}]


## Other ideas about the datasets

When I did some research on OSM and formats or tags used, I stumbled upon some website with local OSM groups and there was also one in Paderborn (which obviously isn't active anymore, but it existed). Furthermore, as shown above, 906 users worked on the data. For that reason I expected to have only a few inconsistencies and problems. That was only partly true.

#### Phone numbers
As mentioned above, the format of the phone numbers was inconsistent. I think I have shown that it is really easy to make them consistent and especially reduce wrongly formatted numbers. I am surprised that nobody did that yet. 


#### Possible improvements
Additionally, I think the focus people had when creating the data was not that optimal. It's funny to have detailed data about benches, but I can't think of any practical need for that. In contrast, there are 158 restaurants without any information about opening hours, which is definitely a more important information. It might be difficult to find a good and consistent format, but I think it would be worth implementing and standardizing it.